In [1]:
import numpy as np
import pandas as pd
import data_prep 
import TensorRecommend
import os

#geting this path to work on any computer
base_path = os.getcwd()
games_info_path = os.path.join(base_path, "..", "data", "raw", "games_detailed_info2025.csv")
bgg_reviews_path = os.path.join(base_path, "..", "data", "raw", "bgg-26m-reviews.csv")

games_info = pd.read_csv(games_info_path)
games_info.rename(columns={'id': 'ID'}, inplace=True) #renaming column to match with reviews dataset for merging
bgg_reviews = pd.read_csv(bgg_reviews_path)

dataprep = data_prep.DataPreprocessor(games_info, bgg_reviews)
dataprep.discretize_column('usersrated', n_bins=10, percent=0.01, drop_original=True )
dataprep.discretize_column('yearpublished', n_bins=10, drop_original=True)

,Unnamed: 0,type,ID,thumbnail,image,alternate,description,minplayers,maxplayers,suggested_num_players,...,Party Game Rank,Thematic Rank,War Game Rank,Customizable Rank,Children's Game Rank,RPG Item Rank,Accessory Rank,name,usersrated_bin,yearpublished_bin
0,0,boardgame,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,"['Catan', 'Catan (Колонизаторы)', 'Catan telep...","In CATAN (formerly The Settlers of Catan), pla...",3,4,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CATAN,9,0
1,1,boardgame,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,"['Carcassonne Jubilee Edition', 'Carcassonne: ...",Carcassonne is a tile placement game in which ...,2,5,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carcassonne,9,1
2,2,boardgame,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,"['EPIZOotic', 'Pandemia', 'Pandemia 10 Anivers...","In Pandemic, several virulent diseases have br...",2,4,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pandemic,9,2
3,3,boardgame,68448,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,"['7 csoda', '7 Cudów Świata', '7 divů světa', ...",You are the leader of one of the 7 great citie...,2,7,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7 Wonders,9,3
4,4,boardgame,167791,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZ...,https://cf.geekdo-images.com/wg9oOLcsKvDesSUdZ...,"['A Mars terraformálása', 'Mars: Teraformace',...","In the 2400s, mankind begins to terraform the ...",1,5,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Terraforming Mars,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,272,boardgame,336986,https://cf.geekdo-images.com/EvGtnsBDcfnKiqSiX...,https://cf.geekdo-images.com/EvGtnsBDcfnKiqSiX...,"['Draci v práci', 'Lángművesek', 'Вогнетвір', ...","Artisan dragons, the smaller and magically tal...",1,5,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Flamecraft,0,9
273,273,boardgame,161417,https://cf.geekdo-images.com/7zH6Kt5ebWEixjDGm...,https://cf.geekdo-images.com/7zH6Kt5ebWEixjDGm...,"['7 Rosso', 'Red', 'Red 7', 'Red Seven', 'Ruz7...",The rules of &quot;Red&quot; are simple: highe...,2,4,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Red7,0,5
274,274,boardgame,172818,https://cf.geekdo-images.com/U0wpvRmBe65e4vwGf...,https://cf.geekdo-images.com/U0wpvRmBe65e4vwGf...,"['Above and Below: Les royaumes du dessous', '...",Your last village was ransacked by barbarians....,2,4,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,117.0,NaN,NaN,NaN,NaN,NaN,Above and Below,0,6
275,275,boardgame,221194,https://cf.geekdo-images.com/hLJv1A1TwrMrLLIJh...,https://cf.geekdo-images.com/hLJv1A1TwrMrLLIJh...,"['Ilha dos Dinossauros', 'Wyspa dinozaurów', '...","In Dinosaur Island, players will have to colle...",1,4,"[{'@numplayers': '1', 'result': [{'@value': 'B...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dinosaur Island,0,7


In [2]:
data = dataprep.merge_datasets(featires_to_add = ['yearpublished_bin', 'usersrated_bin', 'minplayers', 'maxplayers'])
#dropping columns that are not needed for training and dropping rows with missing values
data.drop(columns=['Unnamed: 0', 'comment', 'name'], inplace=True)
data.dropna(inplace=True)

In [3]:
data.info()

<class 'pandas.DataFrame'>
Index: 9365559 entries, 0 to 9365581
Data columns (total 7 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user               str    
 1   rating             float64
 2   ID                 int64  
 3   yearpublished_bin  int64  
 4   usersrated_bin     int64  
 5   minplayers         int64  
 6   maxplayers         int64  
dtypes: float64(1), int64(5), str(1)
memory usage: 571.6 MB


# Demo model

In [4]:
"""
data_entries - number of rows in the final dataset
num_users - number of unique users in the dataset
num_games - number of unique games in the dataset
num_features - number of unique values for each feature in the dataset
user2idx - dictionary mapping user to index value in tensor
game2idx - dictionary mapping game to index value in tensor
feature2idx - list of dictionaries mapping feature values to index values in tensor for each feature
"""

'\ndata_entries - number of rows in the final dataset\nnum_users - number of unique users in the dataset\nnum_games - number of unique games in the dataset\nnum_features - number of unique values for each feature in the dataset\nuser2idx - dictionary mapping user to index value in tensor\ngame2idx - dictionary mapping game to index value in tensor\nfeature2idx - list of dictionaries mapping feature values to index values in tensor for each feature\n'

In [5]:
data_entries, num_users, num_games, num_features, user2idx, game2idx, feature2idx = dataprep.data_to_tuple(data, feature_col=['usersrated_bin', 'minplayers'])

In [6]:
data_train, data_test = dataprep.train_test_split(data_entries, test_size=0.6)

Traning time!

In [7]:
epochs = 1 #number of epochs for training
k = 5 #number of latent factors
lambda_ = 0.1 #regularization parameter
eta = 0.1 #learning rate
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=eta, data_entries=data_train, num_users=num_users, 
                                             num_items=num_games, num_features=num_features)


for epoch in range(epochs):
    tessst.train_epoch(t=epoch+1, initial_lr=eta, max_norm=5.0)


KeyboardInterrupt: 

In [ ]:
# Save the model so that this works on any computer without needing to retrain
base_path = os.getcwd()
model_path = os.path.join(os.path.dirname(base_path), "model_26_2")
os.makedirs(model_path, exist_ok=True)
tessst.save_model(path=model_path)

In [ ]:
"""
    Keep only the records in the test set where the user and item are already in the train set.
    This is important because the model cannot give recommendations for users or games it hasn't seen during training.
def filter_test_set(train_entries, test_entries):

    train_users = set()
    train_items = set()

    # izvučemo sve user i item indekse iz traina
    for entry in train_entries:
        u_idx, m_idx, *_ = entry
        train_users.add(u_idx)
        train_items.add(m_idx)

    # filtriramo test
    filtered_test = [
        entry for entry in test_entries
        if entry[0] in train_users and entry[1] in train_items
    ]

    return filtered_test


filtered_test = filter_test_set(data_train, data_test)
"""

In [ ]:
epochs = 5
k = 5
lambda_ = 0.1
path_to_model = os.path.join(os.path.dirname(base_path), "model_26_2")
tessst = TensorRecommend.TensorRecommend(k, lambda_=lambda_, eta=0.1, data_entries=data_train, num_users=num_users, 
                                             num_items=num_games, num_features=num_features, loading=True, path=path_to_model)

In [ ]:
import numpy as np
#Evaluate the model using RMSE on the filtered test set
def evaluate_rmse(model, test_entries):
    errors = []

    for entry in test_entries:
        pred = model.predict(entry)
        r = entry[-1]
        errors.append((r - pred) ** 2)

    return np.sqrt(np.mean(errors))

In [ ]:
import random

random.seed(42)
# evaluation on 20% of the filtered test set for faster evaluation
n_20 = int(0.2 * len(filtered_test))
subset_20 = random.sample(filtered_test, n_20)

print(evaluate_rmse(tessst, subset_20))

2.328291784557648


In [ ]:
tessst.top_games(tessst.U[0])

[(23, np.float64(9.904908013108011)),
 (38, np.float64(9.754997343970532)),
 (174, np.float64(9.721251379169761)),
 (144, np.float64(9.718109477804626)),
 (30, np.float64(9.690383559952249)),
 (78, np.float64(9.624102480846275)),
 (42, np.float64(9.62403863121237)),
 (37, np.float64(9.495872843909197)),
 (89, np.float64(9.458724056353415)),
 (121, np.float64(9.422517311437744))]

In [ ]:
user_vec = tessst.U[user_idx]
top = tessst.top_games(user_vec, top_n=10)
top_10_names =dataprep.top_games_for_user_name_wraper(top,0)
print(top_10_names)